# Crawling PTA

In [1]:
import pandas as pd
!pip install requests
!pip install html5lib
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=494df939a052f7b3284418abaad82dc964888ef504063ba9cd37fd29eb13ba0e
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
# constant
LAST_INDEX = -1
FIRST_INDEX = 1
INCREMENT_BY_ONE = 1

In [3]:
# import library
from bs4 import BeautifulSoup
import requests

def request_url(url):
    return requests.get(url)

def request_header_url(header, url_website):
    headers = {'User-Agent': header}
    return requests.get(url=url_website, headers=headers)

def parse_website(request):
    """Use html5lib library to parse"""
    return BeautifulSoup(request.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib    

def prettify_web_structure(parsed_page):
    parsed_page.prettify()

def get_content_table(web_element, tag, attributes):
    return web_element.find(tag, attrs = attributes)

In [4]:
# max_pages = 172 # total halaman di website PTA, terbaru februari 2023

r = request_header_url("Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1", "https://pta.trunojoyo.ac.id/c_search/byprod/10")
soup = parse_website(r)
prettify_web_structure(soup)
table = get_content_table(soup, "div", {"id":"wrapper"})

pagination = table.findAll("a", attrs = {"class":"pag_button"})
total_pages = int(pagination[LAST_INDEX]["href"].split("/")[LAST_INDEX])

papers = []
for pages in range(FIRST_INDEX, total_pages + INCREMENT_BY_ONE):

    url_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"
    id_prodi = url_link.split("/")[LAST_INDEX]
    nama_prodi = ""
    if (id_prodi == "10"):
        nama_prodi = "Teknik Informatika"

    r_pages = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", f"{url_link}/{pages}")
    soup_pages = parse_website(r_pages)
    prettify_web_structure(soup_pages)
    table_pages = get_content_table(soup_pages, "div", {"id":"wrapper"})

    for article_row in table_pages.findAll("a", attrs = {"class":"gray button"}):
        r_article_row = request_header_url("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", article_row["href"])
        soup_article_row = parse_website(r_article_row)
        prettify_web_structure(soup_article_row)
        table_article_row = get_content_table(soup_article_row, "li", {"data-id":"id-1", "data-cat":"#luxury"})
        article_row_title = table_article_row.find('a', attrs = {"class":"title", "href":"#"}).string
        article_row_abstract = table_article_row.find('p', attrs = {"align":"justify"}).string
        nim = str(article_row["href"]).split("/")[LAST_INDEX]
        papers.append([
            nim,
            article_row_title, 
            article_row_abstract,
            nama_prodi
        ])

In [5]:
df = pd.DataFrame(papers, columns=["NIM", "Judul", "Abstrak", "Program Studi"])
df

,NIM,Judul,Abstrak,Program Studi
0,040411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika
1,040411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika
2,040411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika
3,070411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika
4,080411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika
...,...,...,...,...
853,160411100032,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Investasi saham selama ini memiliki resiko ker...,Teknik Informatika
854,160411100182,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Information Retrieval (IR) merupakan pengambil...,Teknik Informatika
855,160411100077,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Klasifikasi citra merupakan proses pengelompok...,Teknik Informatika
856,160411100084,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Identifikasi atribut pejalan kaki merupakan sa...,Teknik Informatika


In [6]:
import os  
df.to_csv('/content/PTA.csv')  
# pd.read_csv('/content/drive/MyDrive/prosaindata/tugas/PTA.csv')